Разобраться с PyTorch-LifeStream сразу не получилось до конца. Да как водится времени не хватило

In [26]:
import os
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import types as T
from ptls.preprocessing import PysparkDataPreprocessor


data_path = 'data/'

spark_conf = pyspark.SparkConf()
spark_conf.setMaster("local[*]").setAppName("PysparkDataPreprocessor")
spark_conf.set("spark.driver.cores", 30) #Задаем только в Cluster Mode 
spark_conf.set("spark.driver.maxResultSize", "4g")
spark_conf.set("spark.executor.memory", "120g")
spark_conf.set("spark.executor.memoryOverhead", "4g")
spark_conf.set("spark.driver.memory", "120g")
spark_conf.set("spark.driver.memoryOverhead", "4g")
spark_conf.set("spark.cores.max", "24")
spark_conf.set("spark.sql.shuffle.partitions", "200")
spark_conf.set("spark.local.dir", "../../spark_local_dir")


spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
spark.sparkContext.getConf().getAll()

[('spark.driver.memory', '120g'),
 ('spark.driver.memoryOverhead', '4g'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/jdk.internal.ref=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false'),
 ('spark.app.startTime', '171

In [2]:
from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.iterable_processing.iterable_seq_len_limit import ISeqLenLimit
from ptls.data_load.iterable_processing.to_torch_tensor import ToTorch
from ptls.data_load.iterable_processing.feature_filter import FeatureFilter
from ptls.nn import TrxEncoder, RnnSeqEncoder
from ptls.frames.coles import CoLESModule
from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.frames.coles import ColesIterableDataset
from ptls.frames.coles.split_strategy import SampleSlices
from ptls.frames import PtlsDataModule
from ptls.data_load.utils import collate_feature_dict
from ptls.data_load.iterable_processing_dataset import IterableProcessingDataset

from tqdm.auto import tqdm
import lightgbm as ltb

/opt/conda/envs/colab/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/envs/colab/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [3]:
# Открыть тестовые файлы с помощью spark
geo_test = spark.read.parquet("data/geo_test.parquet")
# geo_train_sp = spark.read.parquet("data/geo_train.parquet")
trx_test = spark.read.parquet("data/trx_test.parquet")
# target_test = spark.read.parquet("data/test_target_b.parquet")

In [3]:
# Открыть тренировочные файлы с помощью spark
# geo_train = spark.read.parquet("data/geo_train.parquet")
trx_train = spark.read.parquet("data/trx_train.parquet")
# target_train = spark.read.parquet("data/train_target.parquet")

In [6]:
target_train.printSchema()

root
 |-- mon: string (nullable = true)
 |-- target_1: integer (nullable = true)
 |-- target_2: integer (nullable = true)
 |-- target_3: integer (nullable = true)
 |-- target_4: integer (nullable = true)
 |-- client_id: string (nullable = true)



In [6]:
target_test_20.printSchema()

root
 |-- mon: string (nullable = true)
 |-- target_1: integer (nullable = true)
 |-- target_2: integer (nullable = true)
 |-- target_3: integer (nullable = true)
 |-- target_4: integer (nullable = true)
 |-- client_id: string (nullable = true)



In [7]:
trx_test_20.printSchema()

root
 |-- event_time: timestamp (nullable = true)
 |-- amount: float (nullable = true)
 |-- client_id: string (nullable = true)
 |-- event_type: integer (nullable = true)
 |-- event_subtype: integer (nullable = true)
 |-- currency: integer (nullable = true)
 |-- src_type11: integer (nullable = true)
 |-- src_type12: integer (nullable = true)
 |-- dst_type11: integer (nullable = true)
 |-- dst_type12: integer (nullable = true)
 |-- src_type21: integer (nullable = true)
 |-- src_type22: integer (nullable = true)
 |-- src_type31: integer (nullable = true)
 |-- src_type32: integer (nullable = true)



In [8]:
geo_test_20.printSchema()

root
 |-- client_id: string (nullable = true)
 |-- event_time: timestamp (nullable = true)
 |-- geohash_4: integer (nullable = true)
 |-- geohash_5: integer (nullable = true)
 |-- geohash_6: integer (nullable = true)



Обучение модели для преобразования данных для ptls и трансформация данных через Pyspark

In [4]:
from ptls.preprocessing import PysparkDataPreprocessor

trx_preprocessor = PysparkDataPreprocessor(
    col_id='client_id',
    col_event_time='event_time',
    event_time_transformation='dt_to_timestamp',
    cols_numerical=["event_type",
                   "event_subtype",
                   "currency",
                   "src_type11",
                   "src_type12",
                   "dst_type11",
                   "dst_type12",
                   "src_type21",
                   "src_type22",
                   "src_type31",
                   "src_type32"],
    cols_identity="amount",
)

geo_preprocessor = PysparkDataPreprocessor(
    col_id='client_id',
    col_event_time='event_time',
    event_time_transformation='dt_to_timestamp',
    cols_numerical=['geohash_4',
                    'geohash_5',
                    'geohash_6']
)

target_preprocessor = PysparkDataPreprocessor(
    col_id="client_id",
    col_event_time="mon",
    event_time_transformation="dt_to_timestamp",
    cols_identity=["target_1", "target_2", "target_3", "target_4"]

)

In [6]:
%%time

proc_test = trx_preprocessor.fit_transform(trx_test).persist()
proc_geo_test = geo_preprocessor.fit_transform(geo_test).persist()

CPU times: user 16.9 ms, sys: 2.75 ms, total: 19.7 ms
Wall time: 158 ms


24/06/16 10:41:37 WARN CacheManager: Asked to cache already cached data.


In [5]:
%%time

proc_train = trx_preprocessor.fit_transform(trx_train).persist()
proc_geo_train = geo_preprocessor.fit_transform(geo_train).persist()

CPU times: user 14.4 ms, sys: 2.83 ms, total: 17.2 ms
Wall time: 520 ms


24/06/16 10:58:38 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


***В виду того, что преобразование из формата spark в pandas очень затратный по времени, решил записать результат работы в parquet и потом прочитать в df***

In [9]:
# Запись данных после препроцессинга в формат parquet
proc_test.write.parquet('data/test.parquet', mode='overwrite')
proc_geo_test.write.parquet('data/geo_test.parquet', mode='overwrite')

In [6]:
# Запись данных после препроцессинга в формат parquet
geo_train.write.parquet('data/proc_geo_train.parquet', mode='overwrite')
proc_train.write.parquet('data/train.parquet', mode='overwrite')

In [1]:
import os
os.environ["OMP_NUM_THREADS"] = "1"

import pandas as pd
import numpy as np
import torch
from functools import partial
import pytorch_lightning as pl
import warnings
warnings.filterwarnings("ignore")
import datetime

from ptls.data_load.datasets import MemoryMapDataset
from ptls.data_load.iterable_processing.iterable_seq_len_limit import ISeqLenLimit
from ptls.data_load.iterable_processing.to_torch_tensor import ToTorch
from ptls.data_load.iterable_processing.feature_filter import FeatureFilter
from ptls.nn import TrxEncoder, RnnSeqEncoder
from ptls.frames.coles import CoLESModule
from ptls.data_load.iterable_processing import SeqLenFilter
from ptls.frames.coles import ColesIterableDataset
from ptls.frames.coles.split_strategy import SampleSlices
from ptls.frames import PtlsDataModule
from ptls.data_load.utils import collate_feature_dict
from ptls.data_load.iterable_processing_dataset import IterableProcessingDataset

from tqdm.auto import tqdm
import lightgbm as ltb

/opt/conda/envs/colab/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
test = pd.read_parquet("data/preproc/test.parquet")
train = pd.read_parquet("data/preproc/train.parquet")

In [3]:
def uniq_values(val):
    set_list = set()
    for x in val:
        if x is not None:
            for y in list(x):
                set_list.add(y)
    return set_list

columns_list = ["event_type","event_subtype",
                   "currency",
                   "src_type11",
                   "src_type12",
                   "dst_type11",
                   "dst_type12",
                   "src_type21",
                   "src_type22",
                   "src_type31",
                   "src_type32"
                   ]

len_dict = {}
for x in columns_list:
    y = str(x) + '_in'
    len_dict[y] = len(uniq_values(test[x]))

In [6]:
len_dict

{'event_type_in': 56,
 'event_subtype_in': 58,
 'currency_in': 55,
 'src_type11_in': 266074,
 'src_type12_in': 266282,
 'dst_type11_in': 295587,
 'dst_type12_in': 295829,
 'src_type21_in': 50121,
 'src_type22_in': 33841,
 'src_type31_in': 62101,
 'src_type32_in': 60246}

**Обработка датасета:**

- Транзакции, у которых размер < min_seq_len выкидываются
- Транзакции, у которых длина > max_seq_len, обрезаются и конвертируются в torch.tensor
- Не нужные для CoLES фичи удаляются

In [4]:
train_m = MemoryMapDataset(
    data=train.to_dict("records"),
    i_filters=[
        FeatureFilter(drop_feature_names=['client_id', 'target_1', 'target_2', 'target_3', 'target_4']),
        SeqLenFilter(min_seq_len=32),
        ISeqLenLimit(max_seq_len=1024),
        ToTorch()
    ]
)

test_m = MemoryMapDataset(
    data=test.to_dict("records"),
    i_filters=[
        FeatureFilter(drop_feature_names=['client_id', 'target_1', 'target_2', 'target_3', 'target_4']),
        SeqLenFilter(min_seq_len=32),
        ISeqLenLimit(max_seq_len=1024),
        ToTorch()
    ]
)

In [5]:
train_ds = ColesIterableDataset(
    data=train_m,
    splitter=SampleSlices(
        split_count=5,
        cnt_min=32,
        cnt_max=90
    )
)

valid_ds = ColesIterableDataset(
    data=test_m,
    splitter=SampleSlices(
        split_count=5,
        cnt_min=32,
        cnt_max=90
    )
)

In [6]:
train_dl = PtlsDataModule(
    train_data=train_ds,
    train_num_workers=16,
    train_batch_size=64,
    valid_data=valid_ds,
    valid_num_workers=16,
    valid_batch_size=64
)

# Model

- numeric_values обрабатываются как BatchNorm+Linear
- embedidngs - nn.Embedidngs

In [7]:
trx_encoder_params = dict(
    embeddings_noise=0.003,
    numeric_values={'amount': 'log'},
    embeddings={
        "event_type": {'in': len_dict['event_type_in'], "out": 12},
        "event_subtype": {'in': len_dict["event_subtype_in"], "out": 12},
        'src_type11': {'in': len_dict["src_type11_in"], 'out': 12},
        'src_type12': {'in': len_dict["src_type12_in"], 'out': 12},
        'dst_type11': {'in': len_dict["dst_type11_in"], 'out': 12},
        'dst_type12': {'in': len_dict["dst_type12_in"], 'out': 12},
        'src_type22': {'in': len_dict["src_type22_in"], 'out': 12},
        'src_type31': {'in': len_dict["src_type31_in"], 'out': 12},
        'src_type32': {'in': len_dict["src_type32_in"], 'out': 12},
        }
)

- **TrxEncoder** - обрабатывает каждую тразнакцию (строит для неё эмбеддиг)
- **SeqEncoder** - обрабатывает последовательность

In [8]:
seq_encoder = RnnSeqEncoder(
    trx_encoder=TrxEncoder(**trx_encoder_params),
    hidden_size=32,
    type='gru',
)

In [9]:
model = CoLESModule(
    seq_encoder=seq_encoder,
    optimizer_partial=partial(torch.optim.Adam, lr=0.001),
    lr_scheduler_partial=partial(torch.optim.lr_scheduler.StepLR, step_size=3, gamma=0.9025)
)

In [11]:
trainer = pl.Trainer(
    precision="16-mixed",
    max_epochs=2,
    # limit_val_batches=000,
    accelerator = "gpu",
    devices = 1,
    enable_progress_bar=True,
    gradient_clip_val=0.5,
    logger=pl.loggers.TensorBoardLogger(
        save_dir='.data/logdir',
        name='baseline_result'
    ),
    callbacks=[
        pl.callbacks.LearningRateMonitor(logging_interval='step'),
        pl.callbacks.ModelCheckpoint(every_n_train_steps=2000, save_top_k=-1),
    ]
)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
# trainer = pl.Trainer(
#     precision="16-mixed",
#     max_epochs=2,
#     accelerator = "gpu",
#     devices = 1,
#     enable_progress_bar=True,
#     gradient_clip_val=0.5)

In [12]:
trainer.fit(model, train_dl)

You are using a CUDA device ('NVIDIA GeForce RTX 3090 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type            | Params
-------------------------------------------------------
0 | _loss              | ContrastiveLoss | 0     
1 | _seq_encoder       | RnnSeqEncoder   | 15.4 M
2 | _validation_metric | BatchRecallTopK | 0     
3 | _head              | Head            | 0     
-------------------------------------------------------
15.4 M    Trainable params
0         Non-trainable params
15.4 M    Total params
61.499    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=2000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.


In [13]:
torch.save(model.state_dict(), 'data/model.pt')

In [19]:
class GetSplit(IterableProcessingDataset):
    def __init__(
        self,
        start_month,
        end_month,
        year=2022,
        col_id='client_id',
        col_time='event_time'
    ):
        super().__init__()
        self.start_month = start_month
        self.end_month = end_month
        self._year = year
        self._col_id = col_id
        self._col_time = col_time

    def __iter__(self):
        for rec in self._src:
            for month in range(self.start_month, self.end_month+1):
                features = rec[0] if type(rec) is tuple else rec
                features = features.copy()

                if month == 12:
                    month_event_time = datetime(self._year + 1, 1, 1).timestamp()
                else:
                    month_event_time = datetime(self._year, month + 1, 1).timestamp()

                year_event_time = datetime(self._year, 1, 1).timestamp()

                mask = features[self._col_time] < month_event_time

                for key, tensor in features.items():
                    if key.startswith('target'):
                        features[key] = tensor[month - 1].tolist()
                    elif key != self._col_id:
                        features[key] = tensor[mask]

                features[self._col_id] += '_month=' + str(month)

                yield features

def collate_feature_dict_with_target(batch, col_id='client_id', targets=False):
    batch_ids = []
    target_cols = []
    for sample in batch:
        batch_ids.append(sample[col_id])
        del sample[col_id]

        if targets:
            target_cols.append([sample[f'target_{i}'] for i in range(1, 5)])
            del sample['target_1']
            del sample['target_2']
            del sample['target_3']
            del sample['target_4']

    padded_batch = collate_feature_dict(batch)
    if targets:
        return padded_batch, batch_ids, target_cols
    return padded_batch, batch_ids


class InferenceModuleMultimodal(pl.LightningModule):
    def __init__(self, model, pandas_output=True, drop_seq_features=True, model_out_name='out'):
        super().__init__()

        self.model = model
        self.pandas_output = pandas_output
        self.drop_seq_features = drop_seq_features
        self.model_out_name = model_out_name

    def forward(self, x):
        x_len = len(x)
        if x_len == 3:
            x, batch_ids, target_cols = x
        else:
            x, batch_ids = x

        out = self.model(x)
        if x_len == 3:
            target_cols = torch.tensor(target_cols)
            x_out = {
                'client_id': batch_ids,
                'target_1': target_cols[:, 0],
                'target_2': target_cols[:, 1],
                'target_3': target_cols[:, 2],
                'target_4': target_cols[:, 3],
                self.model_out_name: out
            }
        else:
            x_out = {
                'client_id': batch_ids,
                self.model_out_name: out
            }
        torch.cuda.empty_cache()

        if self.pandas_output:
            return self.to_pandas(x_out)
        return x_out

    @staticmethod
    def to_pandas(x):
        expand_cols = []
        scalar_features = {}

        for k, v in x.items():
            if type(v) is torch.Tensor:
                v = v.cpu().numpy()

            if type(v) is list or len(v.shape) == 1:
                scalar_features[k] = v
            elif len(v.shape) == 2:
                expand_cols.append(k)
            else:
                scalar_features[k] = None

        dataframes = [pd.DataFrame(scalar_features)]
        for col in expand_cols:
            v = x[col].cpu().numpy()
            dataframes.append(pd.DataFrame(v, columns=[f'{col}_{i:04d}' for i in range(v.shape[1])]))

        return pd.concat(dataframes, axis=1)

In [30]:
from ptls.preprocessing import PandasDataPreprocessor
target_train = pd.read_parquet("data/train_target.parquet")

target_preprocessor = PandasDataPreprocessor(
    col_id="client_id",
    col_event_time="mon",
    event_time_transformation="dt_to_timestamp",
    cols_identity=["target_1", "target_2", "target_3", "target_4"],
    # return_records=False,
)

processed_target = target_preprocessor.fit_transform(target_train)

In [34]:
for i in processed_target:
    del i["event_time"]

In [40]:
processed_target[:1]

[{'client_id': '000006265d27d1166ed67506682be7380007a5bead4362f0a9795f7d97fb08e3',
  'target_1': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32),
  'target_2': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32),
  'target_3': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], dtype=torch.int32),
  'target_4': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32)}]

In [37]:
target_df = pd.DataFrame(processed_target)

In [48]:
%%time

train = MemoryMapDataset(
    data=train.merge(target_df, on="client_id", how="inner").to_dict("records"),
    i_filters=[
        ISeqLenLimit(max_seq_len=4096),
        FeatureFilter(keep_feature_names=['client_id', 'target_1', 'target_2', 'target_3', 'target_4']),
        GetSplit(start_month=1, end_month=12),
        ToTorch(),
    ]
)

test = MemoryMapDataset(
    data=test.to_dict("records"),
    i_filters=[
        ISeqLenLimit(max_seq_len=4096),
        FeatureFilter(keep_feature_names=['client_id', 'target_1', 'target_2', 'target_3', 'target_4']),
        ToTorch(),
    ]
)

TypeError: 'module' object is not callable

Наверное надо было получить эмбединги и обучать ml модели по классике